# トレーニング スクリプトをコマンド ジョブとして実行する

Azure Machine Learning 用の Python SDK を使って、スクリプトをコマンド ジョブとして送信できます。 ジョブを使用すると、機械学習モデルのトレーニング時に入力パラメーターと出力を簡単に追跡できます。

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は、既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、スタジオの右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## コマンド ジョブを開始する

以下のセルを実行して、糖尿病を予測する分類モデルをトレーニングします。 モデルは、**src** フォルダーにある **train-model-parameters.py** スクリプトを実行してトレーニングされます。 **diabetes.csv** ファイルをトレーニング データとして使用します。 

- `code`: 実行するスクリプトを含むフォルダーを指定します。
- `command`: 正確に実行する内容を指定します。
- `environment`: コマンドを実行する前にコンピューティングにインストールする必要があるパッケージを指定します。
- `compute`: コマンドの実行に使用するコンピューティングを指定します。
- `display_name`: 個々のジョブの名前。
- `experiment_name`: ジョブが属する実験の名前。

ターミナルでスクリプトをテストするために使用されるコマンドは、次のジョブの構成のコマンドと同じであることにご注意ください。 

In [ ]:
# enter details of your AML workspace
subscription_id = "YOUR-SUBSCRIPTION-ID"
resource_group = "rg-dp100-labs"
workspace = "mlw-dp100-labs"

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)